In [ ]:
import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models

##### Text Cleaning

In [ ]:
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif (tag[1][:2] == "VB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif (tag[1][:2] == "RB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif (tag[1][:2] == "JJ"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

def pre_processing(sentence):
    return remove_stopwords(tokenize_sentence(remove_punctuation(sentence)))

In [ ]:
def read_corpus(txt_file):
    
    with open(txt_file, encoding='utf-8') as file:
        # for each doc create list of pre-processed words in that doc
        doc = []

        for line in file:
            if "<doc" in line:  # tag for new doc
                document_words = []  # list of words that will be part of the document
                while True:
                    next_line = file.readline()  # read next line

                    # remove unuseful tags
                    next_line_proc = next_line.replace("<p> ", "").replace("</p>\n", "").replace("/p", "")

                    if "</doc>" in next_line:
                        break
                    # pre-processing steps
                    sentence_words = pre_processing(next_line_proc)
                    document_words.extend(sentence_words)
                doc.append(document_words)
        file.close()

    print("Documents number: ", len(doc))
    
    return doc

##### Topic Modelling

In [ ]:
num_topics = 10

def topic_modelling(doc):
    
    dictionary_LDA = corpora.Dictionary(doc)

    # delete all terms that do NOT appear in at least 3 documents.
    # delete all terms that appear in more than 60% of documents (see filter_extremes official doc).
    dictionary_LDA.filter_extremes(no_below=3, no_above=0.6)

    corpus_freq = [dictionary_LDA.doc2bow(doc) for doc in doc]
    
    lda_model = models.LdaModel(corpus_freq, num_topics=10 ,id2word=dictionary_LDA, passes=3, alpha='auto', chunksize=2000)

    for i in range(0, lda_model.num_topics-1):
        print("Topic ", i, ": \n",[lda_model.print_topic(i)])
    
    return lda_model, corpus_freq

In [ ]:
doc = read_corpus("Travel.txt")

#richiamo topic_modelling, e quindi printo i topic
model, corpus_freq = topic_modelling(doc)

In [ ]:
#mostro il topic per ogni documento
print ("Documents topic list")
for i in range (0, len(corpus_freq)):
    print ("Doc", i, ":", model[corpus_freq[i]])
    